## REMap-SOLAR Pyranometer Data Analysis
### About
This Notebook is used for the Analysis of the Solar Radiation Readings of Pyranometers installed by REMap-SOLAR.

### RUNNING THE TOOL
_**REQUIREMENTS**_
* Python 2.7
* jupyter
* pandas
* numpy
* matplotlib

They are also found in the __requirements.txt__ file.  
You can install these requirements using __pip__ (__sudo pip install -r requirements.txt__ _or_ __pip install -r requirements.txt__)  

_**PROCEDURE**_
1. Enter the input parameters in the cell marked as "_INPUT PARAMETERS_".
2. Run all cells above __SENSOR READING STATS__ (__Cell -> Run All Above__)
3. Go to the cell with the code you want to run (i.e SENSOR READING STATS, MONTHLY STATS, etc)
4. In the Toolbar above: __Cell -> Run Cells__

### LICENSE  
The "REMap-Solar Pyranometer Data Analysis Tool" is provided under the GNU General Public License (GNU-GPL) v3.0

_Copyright (C) 2016 Ben Hur S. Pintor (bhs.pintor@gmail.com)_

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib

__author__ = "Ben Hur S. Pintor"
__contact__ = "bhs.pintor@gmail.com"
__version__ = "0.0.1"

In [ ]:
'''Input file and SUC'''
infile = ""  # path to input file

suc = ""  # (e.g. ADNU, ADZU, etc.) will be used in naming the image plots

'''Directory to save image plots'''
img_savedir = ""

'''Column headers of sensor data'''
solar_names = ['reading count', 'reading time', 'reading']
# solar_names = ['reading_count', 'reading_time', 'wind_spd1', 'gust_spd1', 'wind_spd2', 'gust_spd2', 'wind_dir', 'reading']

'''Recording interval of sensor'''
time_step = 15.0

'''Minimum # of days logging to be considered good monthly data'''
n = 20

In [ ]:
'''Read the input file into a pandas dataframe'''
df = pd.read_csv(infile, 
                 skiprows=1, 
                 parse_dates=[1,], 
                 infer_datetime_format=True, 
                 header=0, 
                 names=solar_names)

df.drop('reading count', axis=1, inplace=True)    # remove the first column (reading_count)

'''Uncomment the lines below for pyranometers installed with REMap-WIND sensors'''
'''Not necessary if you will remove the columns of wind readings beforehand.'''
# df.drop('wind_spd1', axis=1, inplace=True)
# df.drop('gust_spd1', axis=1, inplace=True)
# df.drop('wind_spd2', axis=1, inplace=True)
# df.drop('gust_spd2', axis=1, inplace=True)
# df.drop('wind_dir', axis=1, inplace=True)

df_no_negative = df[df['reading']>0]   # remove negative values

ts = df_no_negative.set_index('reading time')    # set the dataframe into a time series with the index being the reading time
ts_unfiltered = df.set_index('reading time')     # unfiltered time series (with negative values)

'''ALL READINGS (UNFILTERED)'''
all_readings_unfiltered = ts_unfiltered.groupby(pd.TimeGrouper(freq='D'))    # all readings grouped by day
all_readings_by_day = ts.groupby(pd.TimeGrouper(freq='D'))     # non-negative readings grouped by day
all_readings_by_month = ts.groupby(pd.TimeGrouper(freq='M'))   # non-negative readings grouped by month

'''For computations'''
daily_averages = ts.resample('D').sum()/(60.0/time_step)  # daily averages (non-negative)
num_readings_in_day = all_readings_by_day.count()   # number of daily readings (non-negative)

daily_averages.rename(columns={'reading':'daily average'}, inplace=True)
num_readings_in_day.rename(columns={'reading':'num readings in day'}, inplace=True)

daily_averages_in_month = daily_averages.groupby(pd.TimeGrouper(freq='M'))

'''FILTERED READINGS (negative numbers, days with <84 readings)'''
filtered_readings_by_day = pd.concat([num_readings_in_day, daily_averages],
                                     axis=1,
                                     join_axes=[num_readings_in_day.index])

# filter out days with more than 3 hours missing data
filtered_readings_by_day = filtered_readings_by_day[filtered_readings_by_day['num readings in day']>=(21*60.0/time_step)]   
filtered_readings_by_month = filtered_readings_by_day.groupby(pd.TimeGrouper(freq='M'))


In [ ]:
'''DATA'''
'''DAILY'''
d_mean = all_readings_by_day.sum()/(60.0/time_step)                       # daily averages (unfiltered)
d_mean.rename(columns={"reading": "daily average"}, inplace=True)

d_num_readings = all_readings_by_day.count()                              # number of readings in the month
d_num_readings.rename(columns={"reading": "num readings in day"}, inplace=True)

d_min = all_readings_by_day.min()                                        # minimum daily average in the month
d_min.rename(columns={"reading": "lowest reading in day"}, inplace=True)

d_min_when = all_readings_by_day.idxmin()                                # minimum daily average in the month
d_min_when.rename(columns={"reading": "time of lowest reading"}, inplace=True)

d_max = all_readings_by_day.max()                                        # minimum daily average in the month
d_max.rename(columns={"reading": "highest reading in day"}, inplace=True)

d_max_when = all_readings_by_day.idxmax()                                # minimum daily average in the month
d_max_when.rename(columns={"reading": "time of highest reading"}, inplace=True)


'''DAILY BY MONTH'''     
d_m_mean = d_mean.groupby(pd.TimeGrouper(freq='M')).mean()                  # monthly average (based on all) 
d_m_mean.rename(columns={"reading": "monthly average"}, inplace=True)

d_m_num_readings = all_readings_by_month.count()                            # number of readings in the month
d_m_num_readings.rename(columns={"reading": "num readings in month"}, inplace=True)

d_m_min = all_readings_by_month.min()                                       # lowest reading in the month
d_m_min.rename(columns={"reading": "lowest reading in month"}, inplace=True)

d_m_min_when = all_readings_by_month.idxmin()                               # date/time of lowest reading
d_m_min_when.rename(columns={"reading": "time of lowest reading"}, inplace=True)

d_m_max = all_readings_by_month.max()                                       # highest reading in the month
d_m_max.rename(columns={"reading": "highest reading in month"}, inplace=True)

d_m_max_when =  all_readings_by_month.idxmax()                              # date/time of highest reading
d_m_max_when.rename(columns={"reading": "time of highest reading"}, inplace=True)

'''MONTHLY'''
m_mean = filtered_readings_by_month['daily average'].mean()               # monthly average
m_mean.rename('monthly average', inplace=True)

m_num_readings = filtered_readings_by_month['num readings in day'].sum()  # number of readings in the month
m_num_readings.rename('num readings in month', inplace=True)

m_days_logging = filtered_readings_by_month['daily average'].count()
m_days_logging.rename('days logging', inplace=True)

m_min = filtered_readings_by_month['daily average'].min()                 # lowest daily average in the month
m_min.rename('lowest daily average', inplace=True)

m_min_when = filtered_readings_by_month['daily average'].idxmin()         # date of lowest daily average
m_min_when.rename('date of lowest daily average', inplace=True)

m_max = filtered_readings_by_month['daily average'].max()                 # highest daily average in the month
m_max.rename('highest daily average', inplace=True)

m_max_when = filtered_readings_by_month['daily average'].idxmax()         # day of highest daily average
m_max_when.rename('date of highest daily average', inplace=True)

In [ ]:
d_stats = pd.concat([d_num_readings, d_min, d_min_when, d_max, d_max_when, d_mean],
                   axis=1,
                   join_axes=[d_num_readings.index])

d_stats

In [ ]:
d_m_stats = pd.concat([d_m_num_readings, d_m_min, d_m_min_when, d_m_max, d_m_max_when, d_m_mean],
                   axis=1,
                   join_axes=[d_m_num_readings.index])

print("SENSOR STATS for {}".format(infile))
d_m_stats

In [ ]:
m_stats = pd.concat([m_num_readings, m_days_logging, m_min, m_min_when, m_max, m_max_when, m_mean],
                    axis=1,
                    join_axes=[m_num_readings.index])

# m_stats = pd.concat([m_num_readings, m_days_logging, m_min, m_max, m_mean],
#                     axis=1,
#                     join_axes=[m_num_readings.index])


m_stats.drop(m_stats[m_stats['days logging']<n].index, inplace=True)
print("MONTHLY STATS for {}".format(infile))
m_stats

### PLOTS

In [ ]:
plt.style.use('ggplot')

### ALL READINGS PER MONTH

In [ ]:
keys01 = all_readings_by_month.groups.keys();
keys01.sort()

for key in keys01:
    to_plot = all_readings_by_month.get_group(key)
    if to_plot.count().item() < 1:  #1440
        pass
    
    else:
        fig = plt.figure(figsize=(16,6))
        ax = fig.add_subplot(1,1,1)

        ax.spines['bottom'].set_color('black')
        ax.spines['top'].set_color('black') 
        ax.spines['right'].set_color('black')
        ax.spines['left'].set_color('black')

        ax.tick_params(direction='inout', length=8, width=1, colors='black', top=False, right=False)

        ax.set_title("Readings of {} pyranometer for {}".format(suc, str(key)[:-12]))

        ax.set_ylabel('W/m^2')
        ax.set_ylim(0,1400)
        ax.set_xlabel('Date')

        plt.xticks(rotation=90)
        #with plt.style.context('ggplot'):
        plt.plot(to_plot.index, to_plot, '-')
        #plt.savefig("{}/{}_{}_all.png".format(img_savedir, suc, str(key)[:-12]), bbox_inches='tight')
        plt.show()
        

### DAILY AVERAGE PER MONTH    


In [ ]:
keys02 = all_readings_by_month.groups.keys();
keys02.sort()

for key in keys02:
    to_plot = daily_averages_in_month.get_group(key)
    if to_plot.count().item() < 1:
        pass
    
    else:
        fig = plt.figure(figsize=(16,6))
        ax = fig.add_subplot(1,1,1)

        ax.spines['bottom'].set_color('black')
        ax.spines['top'].set_color('black') 
        ax.spines['right'].set_color('black')
        ax.spines['left'].set_color('black')

        ax.tick_params(direction='inout', length=8, width=1, colors='black', top=False, right=False)

        ax.set_title("Daily Average Readings of {} Pyranometer {}".format(suc, str(key)[:-12]))

        ax.set_ylabel('Wh/m^2-day')
        ax.set_ylim(0,8000)
        ax.set_xlabel('Date')

        plt.xticks(rotation=90)
        #with plt.style.context('ggplot'):
        plt.plot(to_plot.index, to_plot, 'o-')
        #plt.savefig("{}/{}_{}_daily.png".format(img_savedir, suc, str(key)[:-12]), bbox_inches='tight')
        plt.show()

### DAILY READINGS

In [ ]:
#all_readings_day.plot(figsize=(8,5), ylim=(0,1400));

keys03 = all_readings_by_day.groups.keys();
keys03.sort()

for key in keys03:
    to_plot = all_readings_by_day.get_group(key)
    if to_plot.count().item() < 80:
        pass
    
    else:
        fig = plt.figure(figsize=(9,6))
        ax = fig.add_subplot(1,1,1)

        ax.spines['bottom'].set_color('black')
        ax.spines['top'].set_color('black') 
        ax.spines['right'].set_color('black')
        ax.spines['left'].set_color('black')

        ax.tick_params(direction='inout', length=8, width=1, colors='black', top=False, right=False)

        ax.set_title("Readings of {} Pyranometer for {}".format(suc, str(key)[:-9]))

        ax.set_ylabel('W/m^2')
        ax.set_ylim(0,1400)
        ax.set_xlabel('Time')

        plt.xticks(rotation=90)
        #with plt.style.context('ggplot'):
        plt.plot_date(to_plot.index, to_plot, 'o-')
        plt.show()